In [ ]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from pydantic import BaseModel

In [ ]:
# -----------------------------------------
# 1. Definir schema en Pydantic
# -----------------------------------------
class Schema(BaseModel):
    decision: Literal["red", "blue"]
    justification: str


# -----------------------------------------
# 2. LLM Ollama (sin function calling)
# -----------------------------------------
model = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0,
)



# -----------------------------------------
# 4. Prompt que obliga al modelo a dar JSON
# -----------------------------------------
routing_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are an artist that decides if the user is talking about something "
                "related to the color red or blue.\n\n"
                "You MUST respond with a VALID JSON object ONLY.\n"
                "STRICT FORMAT:\n"
                "{{\n"
                '  "decision": "red" | "blue",\n'
                '  "justification": "text"\n'
                "}}\n"
                "Do NOT write markdown. Do NOT add explanations. Only JSON."
            ),
        ),
        ("human", "{query}"),
    ]
)

model = model.with_structured_output(
    Schema,
)

# -----------------------------------------
# 5. Cadena final (prompt -> LLM -> parser)
# -----------------------------------------
pipeline = routing_prompt | model


# -----------------------------------------
# 6. Ejecutar
# -----------------------------------------
response = pipeline.invoke({"query": "A horse running to the sea"})

print(response)

c:\Users\facun\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


decision='blue' justification='The mention of the sea evokes the color blue, which is commonly associated with water.'
